# Funções de perda (*loss functions*)

Neste código iremos analisar diferentes funções de perda (também conhecidas como *loss functions*) que são usadas para avaliar a rede no estado atual.

Funções de perda, também conhecidas como *loss functions*, são muito importantes para o aprendizagem de máquinas, pois servem como uma forma de medir a distância ou a diferença entre a saída prevista de um modelo e o seu valor real, auxiliando então no treino no modelo.

Diversas funções de perda foram propostas ao longo do tempo para diferentes tipos de problemas.
Algumas dessas funções foram propostas para auxiliar no treino de modelos de regressão linear, como as *loss* [L1](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.L2Loss), [L2](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.L1Loss) e [Huber](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.HuberLoss).
Outras foram propostas para serem usadas em problemas de classificação, como a mais comum de todas [Cross-Entropy](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.SoftmaxCrossEntropyLoss).

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1ITV4Ikw0NP39p1KNFkt46LWwnhzLKf7h">
</p>


Esse pequeno bloco de código abaixo é usado somente para instalar o MXNet para CUDA 10. Execute esse bloco somente uma vez e ignore possíveis erros levantados durante a instalação.

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [0]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys, numpy as np
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata
from sklearn.model_selection import train_test_split

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

     |████████████████████████████████| 488.3MB 35kB/s 
     |████████████████████████████████| 13.8MB 26.3MB/s 
ERROR: spacy 2.1.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.54 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


gpu(0)

In [0]:
## carregando dados básicos

# dados sintéticos somente para 
def synthetic_regression_data(w, b, num_examples):
    """generate y = X w + b + noise"""
    X = nd.random.normal(scale=1, shape=(num_examples, len(w)))
    y = nd.dot(X, w) + b
    y += nd.random.normal(scale=0.01, shape=y.shape)
    return X, y

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [0]:
# funções básicas

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(features, labels)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs, type='regression'):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        if type == 'regression':
          print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                % (epoch + 1, train_l_sum / n, test_loss, time.time() - start))
        else:
          print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
                'test acc %.3f, time %.1f sec'
                % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                   test_acc, time.time() - start))

## *Loss* L2

O função de custo chamada L2 (também conhecida como *Mean Squared Error* -- MSE) é, talvez, a função de perda mais simples e comum. 
Essa função é representada simplesmente pela média do quadrado da diferença entre as previsões do modelo e o *ground-truth*.
Essa função nunca terá valores negativos, pois a diferença calculada será sempre elevado à segunda potência.

Formalmente, dado a valor real $y$, e a predição feita pelo modelo $\hat{y}$, a *loss* L2 é definida pela seguinte equação:

$$\mathcal{l}_2^i(w, b) = \frac{1}{2} (\hat{y}^i - y^i)^2 $$

A constante $1/2$ é apenas por conveniência matemática, garantindo que depois de tomarmos a derivada dessa função, o coeficiente constante será de $1$.

A grande vantagem dessa função é que ela garante que o modelo treinado não tenha previsões discrepantes com erros enormes, já que ela atribui maior peso a esses erros devido à parte quadrática da função.
Entretanto, isso gera a desvantagem dessa função de custo, pois se o modelo faz uma única previsão muito ruim, a parte quadrática da função aumenta o erro consideravelmente.
No entanto, em muitos casos práticos, não nos importamos muito com esses poucos valores discrepantes e buscamos um modelo mais abrangente que tenha um bom desempenho na maioria.

Para tentar garantir a qualidade do modelo em todo o conjunto de dados, podemos simplesmente calcular a média das perdas no conjunto de treinamento:

$$\mathcal{L}(w, b) = \frac{1}{n} \sum_i^N \mathcal{l}^{i}_2(w, b) $$

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), a implementação de funções de custo comuns, como a L2, são diretas e muitos simples.

**Um exemplo é mostrado abaixo utilizando o framework MxNet.**

In [0]:
seed = nd.array([2, -3.4])
seed_gt = 4.2
features, labels = synthetic_regression_data(seed, seed_gt, 1000)
  
batch_size = 10
data_iter = load_array(features, labels, batch_size)

# arquitetura super simples
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

loss = gloss.L2Loss()  # loss L2
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# treino
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

for i in range(995, 999):
  y_hat = net(features[i:i+1, :])
  print(y_hat, labels[i], labels[i] - y_hat)

epoch 1, loss: 0.040588
epoch 2, loss: 0.000160
epoch 3, loss: 0.000050

[[6.0668907]]
<NDArray 1x1 @cpu(0)> 
[6.073719]
<NDArray 1 @cpu(0)> 
[[0.00682831]]
<NDArray 1x1 @cpu(0)>

[[-0.45210648]]
<NDArray 1x1 @cpu(0)> 
[-0.46103162]
<NDArray 1 @cpu(0)> 
[[-0.00892514]]
<NDArray 1x1 @cpu(0)>

[[3.819031]]
<NDArray 1x1 @cpu(0)> 
[3.8151767]
<NDArray 1 @cpu(0)> 
[[-0.00385427]]
<NDArray 1x1 @cpu(0)>

[[8.072802]]
<NDArray 1x1 @cpu(0)> 
[8.061257]
<NDArray 1 @cpu(0)> 
[[-0.01154423]]
<NDArray 1x1 @cpu(0)>


## *Loss* L1

A função de custo L1 é apenas ligeiramente diferente da L2, mas fornece curiosamente propriedades quase exatamente opostas!
Essa função é representada pelo valor absoluto da diferença entre as previsões do modelo e o *ground-truth*.

Essa função, assim como o *loss* L2, nunca será negativo, pois neste caso estamos sempre assumindo o valor absoluto dos erros.
Formalmente, dado a valor real (*ground-truth*) $y$, e a predição feita pelo modelo $\hat{y}$, a *loss* L1 é definida pela seguinte equação:

$$\mathcal{l}_1^i(w, b) = \sum_i |\hat{y}^i - y^i| $$

A grande vantagem da função de custo L1 cobre diretamente a desvantagem do *loss* L2.
Em outras palaras, como estamos trabalhando com o valor absoluto, todos os erros serão ponderados na mesma escala linear.
Assim, ao contrário do *loss* L2, não estamos colocando muito peso nos valores com grande discrepância e a função de perda fornece uma medida genérica e uniforme do desempenho do modelo.

Por outro lado, a desvantagem desta função é, para alguns casos, não dar pesos diferentes para específico valores discrepantes. Por exemplo, os erros relativamente grandes provenientes dos *outliers* acabam sendo ponderados exatamente como erros menores. Isso pode resultar em nosso modelo sendo ótimo na maior parte do tempo, mas fazendo algumas previsões muito ruins de vez em quando.

Essa função pode ser facilmente implementada no MXNet, como pode ser visto [aqui](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.L1Loss).

In [0]:
seed = nd.array([2, -3.4])
seed_gt = 4.2
features, labels = synthetic_regression_data(seed, seed_gt, 1000)
  
batch_size = 10
data_iter = load_array(features, labels, batch_size)

# arquitetura super simples
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

loss = gloss.L1Loss()  # loss L1
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# treino
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

for i in range(995, 999):
  y_hat = net(features[i:i+1, :])
  print(y_hat, labels[i], labels[i] - y_hat)

epoch 1, loss: 2.686278
epoch 2, loss: 0.667559
epoch 3, loss: 0.009290

[[6.3623056]]
<NDArray 1x1 @cpu(0)> 
[6.3656797]
<NDArray 1 @cpu(0)> 
[[0.0033741]]
<NDArray 1x1 @cpu(0)>

[[7.00144]]
<NDArray 1x1 @cpu(0)> 
[7.006044]
<NDArray 1 @cpu(0)> 
[[0.00460386]]
<NDArray 1x1 @cpu(0)>

[[0.8502693]]
<NDArray 1x1 @cpu(0)> 
[0.8532924]
<NDArray 1 @cpu(0)> 
[[0.00302309]]
<NDArray 1x1 @cpu(0)>

[[3.6928315]]
<NDArray 1x1 @cpu(0)> 
[3.6847017]
<NDArray 1 @cpu(0)> 
[[-0.00812984]]
<NDArray 1x1 @cpu(0)>


## Huber *Loss* 

Vimos que a função de perda L2 tem certas vantangens (como conseguir aprender *outliers*), enquanto o *loss* L1 tem outros benefícios, como ignorar os *outliers*.
Porém, existe uma forma de combinar e agregar os benefícios das duas?

Sim! A Huber *Loss* oferece o melhor dos dois mundos, equilibrando as funções de perda L1 e L2 juntos. 
Formalmente, dado a valor real (*ground-truth*) $y$, e a predição feita pelo modelo $\hat{y}$, a Huber *Loss* é definida pela seguinte equação:

$$
l_H^i(w, b) = \sum_i \begin{cases}
                                        \frac{1}{2\rho} (\hat{y}^i - y^i)^2, & \text{if } |\hat{y}^i - y^i| < \rho\\
                                        |\hat{y}^i - y^i| - \frac{\rho}{2},  & \text{otherwise}
\end{cases}
$$
, onde $\rho$ é uma constance que define a margem.
O que essa equação essencialmente diz é: para valores de perda menores que $\rho$, use o *loss& L2; para valores de perda maiores que delta, use a função de custo L1.
Isso efetivamente combina o melhor dos dois mundos das duas funções de perda!

O uso da função de custo L1 para valores maiores reduz o peso que colocamos em valores discrepantes para que possamos obter um modelo completo. Ao mesmo tempo, usamos o *loss* L2 para valores menores de perda para manter uma função quadrática próxima ao centro.

Essa função pode ser facilmente implementada no MXNet, como pode ser visto [aqui](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.HuberLoss).

In [0]:
seed = nd.array([2, -3.4])
seed_gt = 4.2
features, labels = synthetic_regression_data(seed, seed_gt, 1000)
  
batch_size = 10
data_iter = load_array(features, labels, batch_size)

# arquitetura super simples
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

loss = gloss.HuberLoss()  # loss Huber
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# treino
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

for i in range(995, 999):
  y_hat = net(features[i:i+1, :])
  print(y_hat, labels[i], labels[i] - y_hat)

epoch 1, loss: 2.245753
epoch 2, loss: 0.361979
epoch 3, loss: 0.000923

[[0.10509491]]
<NDArray 1x1 @cpu(0)> 
[0.10078488]
<NDArray 1 @cpu(0)> 
[[-0.00431003]]
<NDArray 1x1 @cpu(0)>

[[5.689843]]
<NDArray 1x1 @cpu(0)> 
[5.727647]
<NDArray 1 @cpu(0)> 
[[0.03780365]]
<NDArray 1x1 @cpu(0)>

[[5.5963483]]
<NDArray 1x1 @cpu(0)> 
[5.641131]
<NDArray 1 @cpu(0)> 
[[0.04478264]]
<NDArray 1x1 @cpu(0)>

[[4.2930875]]
<NDArray 1x1 @cpu(0)> 
[4.3207493]
<NDArray 1 @cpu(0)> 
[[0.0276618]]
<NDArray 1x1 @cpu(0)>


## *Loss Cross-Entropy*

O função de custo chamada *cross-entropy* ou *log loss* é a mais usada em problemas de classificação.
Essa função de perda, embasada pela teoria da informação, procura penalizar o *loss* baseado somente na classe correta de cada amostra.

Formalmente, dado a valor real $y$, e a predição feita pelo modelo $\hat{y}$, a *loss cross-entropy* é definida pela seguinte equação:

$$\mathcal{l}(w, b) = - \sum_i y_i log~\hat{y}_i $$
, onde $\hat{y}$ é saída normalizada (via [softmax](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Symbol.softmax)) da predição da rede.

Em particular, no somatório apenas um termo será diferente de zero e esse termo será o $log$ da probabilidade (normalizada via [softmax](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Symbol.softmax)) atribuída à classe correta. Intuitivamente, isso faz sentido porque $log (x)$ está aumentando no intervalo (0,1), então $−log (x)$ está diminuindo naquele intervalo.
Por exemplo, se tivermos uma amostra com probabilidade de 0.8 para o rótulo correto, o *loos* será penalizado em apenas 0.09.
Já se tivermos uma probabilidade menor de 0.08, o *loss* será penalizado em 1,09.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), a implementação da função de custo *cross-entropy* é direta.

**Um exemplo é mostrado abaixo utilizando o framework MxNet.**

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential()
net.add(nn.Dense(256, activation="relu"),
        nn.Dense(128, activation="relu"),
        nn.Dense(64, activation="relu"),
        nn.Dense(10))
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.3016, test loss 2.3006, time 3.5 sec
epoch 2, train loss 1.9082, test loss 1.1132, time 4.1 sec
epoch 3, train loss 0.6266, test loss 0.2657, time 4.1 sec
epoch 4, train loss 0.2040, test loss 0.1611, time 4.1 sec
epoch 5, train loss 0.1280, test loss 0.1281, time 4.0 sec
epoch 6, train loss 0.0950, test loss 0.1042, time 4.1 sec
epoch 7, train loss 0.0709, test loss 0.0979, time 4.4 sec
epoch 8, train loss 0.0577, test loss 0.0889, time 4.1 sec
epoch 9, train loss 0.0457, test loss 0.0887, time 4.1 sec
epoch 10, train loss 0.0401, test loss 0.0860, time 4.2 sec
epoch 11, train loss 0.0318, test loss 0.0812, time 4.0 sec
epoch 12, train loss 0.0265, test loss 0.0869, time 4.1 sec
epoch 13, train loss 0.0193, test loss 0.1054, time 4.2 sec
epoch 14, train loss 0.0178, test loss 0.0950, time 4.1 sec
epoch 15, train loss 0.0128, test loss 0.0893, time 4.3 sec
epoch 16, train loss 0.0086, test loss 0.1041, time 4.2 sec
epoch 17, train loss 0.0063, t